In [4]:
import duckdb
import os
import pandas as pd
from datetime import datetime

In [8]:
# 📂 Définition des chemins
DATA_PATH = "../data/cleaned/"
OUTPUT_PATH = "../data/output/user_features_v2.parquet"

In [ ]:
# 📌 Catégories sélectionnées
SELECTED_CATEGORIES = {"electronics", "computers", "sport", "kids"}

def load_parquet_files():
    """Charge tous les fichiers Parquet sous forme d'un dataframe Dask."""
    files = glob.glob(os.path.join(DATA_PATH, "*.parquet"))

    if not files:
        raise FileNotFoundError(f"Aucun fichier .parquet trouvé dans {DATA_PATH}")

    print(f"📂 {len(files)} fichiers trouvés. Chargement avec Dask...")

    # Charger uniquement les colonnes nécessaires
    cols = ["user_id", "event_type", "event_time", "category_code"]
    df = dd.read_parquet(files, columns=cols, engine="pyarrow")

    # 📌 Filtrer uniquement les événements "view" et "purchase"
    df = df[df["event_type"].isin(["view", "purchase"])]

    # 📌 Extraire la catégorie principale (avant le point ".")
    df["category_main"] = df["category_code"].str.split(".").str[0]

    # 📌 Garder uniquement les catégories sélectionnées
    df = df[df["category_main"].isin(SELECTED_CATEGORIES)]

    return df

def generate_user_features():
    """Transforme les données en jeu de caractéristiques par utilisateur."""
    print("🔄 Chargement des données...")
    df = load_parquet_files()

    # 📌 Calcul des métriques par utilisateur
    print("📊 Calcul des métriques par utilisateur...")

    total_events = df.groupby("user_id").size().to_frame("total_events")
    total_views = df[df["event_type"] == "view"].groupby("user_id").size().to_frame("total_views")
    total_purchases = df[df["event_type"] == "purchase"].groupby("user_id").size().to_frame("total_purchases")
    unique_categories = df.groupby("user_id")["category_main"].nunique().to_frame("unique_categories")
    last_event_time = df.groupby("user_id")["event_time"].max().to_frame("last_event_time")

    # 📌 Fusionner toutes les métriques
    user_features = total_events.join([total_views, total_purchases, unique_categories, last_event_time], how="left").fillna(0)

    # 📌 Calcul du taux de transformation
    user_features["conversion_rate"] = user_features["total_purchases"] / user_features["total_views"]
    user_features["conversion_rate"] = user_features["conversion_rate"].fillna(0)

    # 📌 Conversion en Pandas et sauvegarde
    print(f"💾 Sauvegarde du fichier {OUTPUT_PATH}...")
    os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
    user_features.compute().to_parquet(OUTPUT_PATH, engine="pyarrow")

    print("✅ Transformation terminée avec succès !")

if __name__ == "__main__":
    generate_user_features()

🔄 Chargement des données en streaming...
📂 Chargement des fichiers Parquet en streaming avec DuckDB...
📊 Calcul des métriques par utilisateur...
🚀 Exécution de la requête d'agrégation...


RuntimeError: Query interrupted

In [34]:
# 📌 Catégories sélectionnées
SELECTED_CATEGORIES = {"electronics", "computers", "sport", "kids"}

def load_data_with_duckdb():
    """Charge et filtre les données en streaming avec DuckDB pour économiser la mémoire."""
    print("📂 Chargement des fichiers Parquet en streaming avec DuckDB...")

    query = f"""
        SELECT 
            user_id, 
            event_type, 
            event_time,
            price, 
            split_part(category_code, '.', 1) AS category_main
        FROM read_parquet('{DATA_PATH}/*.parquet', hive_partitioning=0)
        WHERE event_type IN ('view', 'purchase','cart')
        AND split_part(category_code, '.', 1) IN ({', '.join(f"'{cat}'" for cat in SELECTED_CATEGORIES)})
    """

    return duckdb.query(query)  # Retourne un objet DuckDB en streaming

def generate_user_features():
    """Transforme les données en jeu de caractéristiques par utilisateur avec DuckDB en optimisant la mémoire."""
    print("🔄 Chargement des données en streaming...")
    df = load_data_with_duckdb()  # Chargement optimisé

    print("📊 Calcul des métriques par utilisateur...")

    # 📌 Agrégation en plusieurs étapes pour réduire la mémoire utilisée
    user_features_query = """
        SELECT 
            user_id,
            COUNT(*) AS total_events,
            COUNT(CASE WHEN event_type = 'view' THEN 1 END) AS total_views,
            COUNT(CASE WHEN event_type = 'cart' THEN 1 END) AS total_carts,
            COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) AS total_purchases,
            SUM(CASE WHEN event_type = 'purchase' THEN price ELSE 0 END) AS total_spent,
            COUNT(DISTINCT category_main) AS unique_categories,
            MAX(event_time) AS last_event_time
        FROM df
        GROUP BY user_id
    """

    user_features = duckdb.query(user_features_query).to_df()

    print("📅 Conversion des dates...")
    # 📌 Conversion des dates (en UTC et naive)
    user_features["last_event_time"] = pd.to_datetime(user_features["last_event_time"]).dt.tz_localize(None)

    print("🔢 Calcul du taux de conversion...")
    # 📌 Calcul du taux de conversion
    user_features["conversion_rate_view"] = user_features["total_purchases"] / user_features["total_views"]
    user_features["conversion_rate_view"].fillna(0, inplace=True)

    print("🔢 Calcul du taux de panier transformé...")
    # 📌 Calcul du taux de conversion
    user_features["conversion_rate_cart"] = user_features["total_purchases"] / user_features["total_carts"]
    user_features["conversion_rate_cart"].fillna(0, inplace=True)

    print("⏳ Calcul du temps depuis le dernier événement...")
    # 📌 Calcul du temps depuis le dernier événement (en jours)
    now_utc = datetime.utcnow()
    user_features["days_since_last_event"] = (
        now_utc - user_features["last_event_time"]
    ).dt.total_seconds() / (3600 * 24)

    # 📌 Nettoyage des valeurs infinies et NaN
    user_features.replace([float("inf"), float("-inf")], None, inplace=True)
    user_features.fillna(0, inplace=True)

    # 📌 Sauvegarde avec compression (pour économiser la RAM)
    print(f"💾 Sauvegarde optimisée du fichier {OUTPUT_PATH}...")
    os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
    user_features.to_parquet(OUTPUT_PATH, engine="pyarrow", compression="snappy")

    print("✅ Transformation terminée avec succès !")

if __name__ == "__main__":
    generate_user_features()

🔄 Chargement des données en streaming...
📂 Chargement des fichiers Parquet en streaming avec DuckDB...
📊 Calcul des métriques par utilisateur...
📅 Conversion des dates...
🔢 Calcul du taux de conversion...
🔢 Calcul du taux de panier transformé...


C:\Users\Pierre\AppData\Local\Temp\ipykernel_24620\241039523.py:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  user_features["conversion_rate_view"].fillna(0, inplace=True)
C:\Users\Pierre\AppData\Local\Temp\ipykernel_24620\241039523.py:58: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a

⏳ Calcul du temps depuis le dernier événement...


C:\Users\Pierre\AppData\Local\Temp\ipykernel_24620\241039523.py:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  user_features.fillna(0, inplace=True)


💾 Sauvegarde optimisée du fichier ../data/output/user_features_v2.parquet...
✅ Transformation terminée avec succès !


In [35]:
# 📂 Chemin du fichier
file_path = "../data/output/user_features_v2.parquet"

# 📖 Chargement et affichage des 10 premières lignes
query = f"SELECT * FROM read_parquet('{file_path}') LIMIT 10"
df = duckdb.query(query).to_df()

df

,user_id,total_events,total_views,total_carts,total_purchases,total_spent,unique_categories,last_event_time,conversion_rate_view,conversion_rate_cart,days_since_last_event
0,547043751,22,22,0,0,0.000000,3,2019-12-07 15:42:17,0.000,0.000000,1923.772912
1,581315515,3,3,0,0,0.000000,1,2019-12-03 10:26:41,0.000,0.000000,1927.992078
2,521025204,35,35,0,0,0.000000,2,2020-04-21 18:26:58,0.000,0.000000,1787.658548
3,515796233,2,2,0,0,0.000000,1,2019-12-03 10:25:34,0.000,0.000000,1927.992854
4,514043743,50,49,1,0,0.000000,3,2020-04-05 15:17:04,0.000,0.000000,1803.790423
5,552655685,16,16,0,0,0.000000,2,2019-12-03 10:32:35,0.000,0.000000,1927.987981
6,516008494,88,80,6,2,419.289993,4,2020-03-17 04:33:12,0.025,0.333333,1823.237553
7,553785560,7,7,0,0,0.000000,1,2020-04-15 18:51:45,0.000,0.000000,1793.641338
8,581357671,4,4,0,0,0.000000,1,2019-12-03 10:33:53,0.000,0.000000,1927.987078
9,512489362,60,60,0,0,0.000000,1,2020-02-16 06:14:01,0.000,0.000000,1853.167541


In [30]:
# Chemin du dossier contenant les fichiers Parquet
dossier_parquet = "../data/cleaned"

# Utilisation d'un caractère générique pour charger tous les fichiers Parquet du dossier
chemin_fichiers = f"{dossier_parquet}/*.parquet"

# Construction de la requête DuckDB pour compter le nombre total d'entrées
requete = f"SELECT * FROM read_parquet('{chemin_fichiers}') LIMIT 10"
df_total = duckdb.query(requete).to_df()

df_total

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,view,1005105,1451229556,construction.tools.light,apple,1302.479980,556695836,ca5eefc5-11f9-450c-91ed-380285a0bc80
1,2019-12-01 00:00:00 UTC,view,22700068,16777546,unknown,force,102.959999,577702456,de33debe-c7bf-44e8-8a12-3bf8421f842a
2,2019-12-01 00:00:01 UTC,view,2402273,553648671,appliances.personal.massager,bosch,313.519989,539453785,5ee185a7-0689-4a33-923d-ba0130929a76
3,2019-12-01 00:00:02 UTC,purchase,26400248,-50331391,computers.peripherals.printer,unknown,132.309998,535135317,61792a26-672f-4e61-9832-7b63bb1714db
4,2019-12-01 00:00:02 UTC,view,20100164,1283457772,apparel.trousers,nika,101.680000,517987650,906c6ca8-ff5c-419a-bde9-967ba8e2233e
5,2019-12-01 00:00:02 UTC,view,100008256,-511704351,accessories.umbrella,ikea,163.559998,542860793,a1bcb550-1065-4769-a80a-0ccb4bcee78d
6,2019-12-01 00:00:02 UTC,view,21400264,-117439751,electronics.clocks,unknown,88.809998,538021416,e88f77cc-e75e-4e9f-9ef6-ef1a302ed50a
7,2019-12-01 00:00:03 UTC,view,1005239,1451229556,construction.tools.light,xiaomi,256.380005,525740700,370e8c88-3d07-41df-9aaa-2adf5a0bf312
8,2019-12-01 00:00:04 UTC,view,5100885,268435735,computers.notebook,jet,20.570000,512509221,4227259f-1c4c-41dc-84b5-9354d864eefa
9,2019-12-01 00:00:04 UTC,view,26205399,-1451229078,construction.components.faucet,unknown,179.160004,553345124,58c692ff-c7a9-4e35-9ec4-58598f1940e0
